## Homework 3 extra: texts clastering

In [1]:
import numpy as np
import jsonlines
import json
import tldextract
from pymystem3 import Mystem
import nltk
from nltk.corpus import stopwords
import re
from tqdm import tqdm
tqdm.pandas(desc="progress-bar")
import time

from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [2]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/ilyabasharov/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/ilyabasharov/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
def month(code):
    
    if code == 1:
        return 'январь'
    elif code == 2:
        return 'февраль'
    elif code == 3:
        return 'март'
    elif code == 4:
        return 'апрель'
    elif code == 5:
        return 'май'
    elif code == 6:
        return 'июнь'
    elif code == 7:
        return 'июль'
    elif code == 8:
        return 'август'
    elif code == 9:
        return 'сентябрь'
    elif code == 10:
        return 'октябрь'
    elif code == 11:
        return 'ноябрь'
    else:
        return 'декабрь'
    
def day(code):
    
    if code == 0:
        return 'понедельник'
    elif code == 1:
        return 'вторник'
    elif code == 2:
        return 'среда'
    elif code == 3:
        return 'четверг'
    elif code == 4:
        return 'пятница'
    elif code == 5:
        return 'суббота'
    else:
        return 'воскресенье'

def year(code):
    
    return str(code)
    

In [4]:
def create_data(file_jsonl, file_json):
    
    tokenizer = nltk.data.load('tokenizers/punkt/russian.pickle')
    russian_stopwords = nltk.corpus.stopwords.words('russian')
    morph = Mystem()
    
    with open(file_json) as file:
        doc_ids = json.load(file)
    
    train, ans, test, dc_ids = [], [], [], []
    
    with jsonlines.open(file_jsonl) as reader:
        for obj in tqdm(reader):
            
            sentanses = tokenizer.tokenize((obj['description'] + obj['title']).lower().strip())
            sentanses = sum([x.split(';') for x in sentanses], [])
            sentanses = [morph.lemmatize(word) for word in sentanses]
            sentanses = [[y for y in x if re.match('[а-яёa-z0-9]', y) and y not in russian_stopwords] for x in sentanses]
            sentanses = sum(sentanses, []) + \
                        [tldextract.extract(obj['url']).domain] + \
                        [month(time.localtime(obj['ts']).tm_mon)] + \
                        [day(time.localtime(obj['ts']).tm_wday)] + \
                        [year(time.localtime(obj['ts']).tm_year)] + \
                        [year(time.localtime(obj['ts']).tm_mday)]
            
            if str(obj['doc_id']) in doc_ids:
                train.append(' '.join(sentanses))
                ans.append(doc_ids[str(obj['doc_id'])])
            else:
                test.append(' '.join(sentanses))
                dc_ids.append(obj['doc_id'])
            
    return train, ans, test, dc_ids

In [5]:
train, ans, test, dc_ids = create_data('cosmo_content_storage_final_cut.jsonl', 'cluster_final_cut_train.json')

88368it [06:28, 227.47it/s]


In [6]:
all_data = train + test

In [7]:
print(len(train), len(ans), len(test))
assert len(ans) == len(train)

26510 26510 61858


In [8]:
X_train, X_test, y_train, y_test = train_test_split(train, ans, test_size=0.1, random_state = 42)

In [9]:
pipeline = Pipeline([('vect', CountVectorizer(max_features = 20000)),
                     ('tfidf', TfidfTransformer())
                    ])
pipeline.fit(all_data)

X_train = pipeline.transform(X_train)
X_test = pipeline.transform(X_test)
Result = pipeline.transform(test)

In [10]:
X_train.shape, X_test.shape, Result.shape

((23859, 20000), (2651, 20000), (61858, 20000))

In [ ]:
clf = SGDClassifier(loss='hinge', penalty='l2',alpha=1e-3, random_state=42, max_iter=50, tol=1e-5, n_jobs = -1)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

print('accuracy %s' % accuracy_score(y_pred, y_test))

In [ ]:
from sklearn.linear_model import LogisticRegression

logreg = LogisticRegression(penalty = 'l2', solver = 'lbfgs', n_jobs=4, C=1e4, multi_class = 'auto')
logreg.fit(X_train, y_train)
y_pred = logreg.predict(X_test)
print('accuracy %s' % accuracy_score(y_pred, y_test))

In [ ]:
result = logreg.predict(Result)

In [ ]:
result.shape, len(dc_ids)

### Result

In [ ]:
with open('solution.txt', 'w') as fout:
    print("doc_id,cat", file=fout)
    for index, dc_id in enumerate(dc_ids):
        print(dc_id, result[index], sep=',', file=fout)